### فاز دوم پروژه
در قسمت دوم پروژه سعی در ردیابی اشیاء و یا افراد داخل ویدیو را داریم. در این بخش از مدل آماده آموزش دیده شده $YOLO$ استفاده میکنیم برای استخراج جعبه های مرزی اشیاء، سپس با استفاده از الگوریتم $SORT$ سعی در ردیابی این اشیاء داریم


In [ ]:
!pip install filterpy

In [ ]:
%cd /content/drive/MyDrive/Project/ # آدرس فایل های پروژه: این آدرس را به محل فایل های خود تغییر دهید
!wget https://pjreddie.com/media/files/yolov3.weights -O config/yolov3.weights

In [3]:
from models import *
from utils import *
import imageio
import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from sort import *
import cv2
from IPython.display import clear_output

### Object Detection
در این بخش وزن های آماده مدل $YOLO$ را لود کرده و با تنظیم هایپر پارامترها در نهایت مدل را روی $Cuda$ برده که از سرعت $GPU$ نیز بهره ببریم

In [7]:
config_path='config/yolov3.cfg'
weights_path='config/yolov3.weights'
class_path='config/coco.names'
img_size=416
conf_thres=0.8
nms_thres=0.4

# Load model and weights
model = Darknet(config_path, img_size=img_size) # تعریف مدل  دیتکشن
model.load_weights(weights_path) # لود کردن وزن های شبکه
model.cuda() # استفاده از جی پی یو باری سرعت بیتشر مدل
model.eval() # مدل از قبل آموزش دیده و تنها باید روی حالت ارزیابی باشد
classes = utils.load_classes(class_path)
Tensor = torch.cuda.FloatTensor

تابع زیر همانطور که از اسم آن پیداست، با گرفتن یک تصویر روی آن پیش پردازش های اولیه را انجام داده و سپس آن را به مدل $Detection$ داده و خروجی های آن که مختصات جعبه های مرزی است را در خروجی میدهد

In [5]:
def detect_image(img):
    # پیش پردازش های اولیه روی تصویر
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])
    
    # تبدیل تصویر به تنسور برای ورودی شبکه
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))
   
   # استفاده از مدل در حالت ارزیابی و دادن تصویر به مدل آموزش دیده شده
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)

    return detections[0] # خروجی جعبه های مرزی داخل تصویر خواهد بود

In [6]:
def convertMillis(millseconds):
    seconds, millseconds = divmod(millseconds, 1000)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    day, hours = divmod(hours, 24)
    seconds = int(seconds + millseconds/10000)
    return f"{int(hours)}:{int(minutes)}:{int(seconds)}"

### Tracking
در کد زیر یک ویدیو در غالب گیف در ورودی داده میشود. سپس بر روی هر یک از فریم های ویدیو پردازش انجام میشود. هر فریم به عنوان یک تصویر جدا در نظر گرفته شده و ابتدا جعبه های مرزی توسط تابع نوشته شده در قسمت قبل استخراج میشود. سپس با استفاده از مدل ترکینگ $SORT$ ترکینگ در راستای فریم های مختلف انجام میشود. در نهایت نیز فریم های پردازش شده ذخیره میشود و در نهایت یک خروجی گیف دیگر خواهیم داشت که در آن اشیاء ئ افراد ترک میشوند

In [ ]:
videopath = 'video/traffic_1.gif' # آدرس ویدیو وروید را اینجا وارد کنید

%pylab inline 

cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

cap = cv2.VideoCapture(videopath) # خواندن ویدو

mot_tracker = Sort() # تعریف مدل ترکینگ

Frames = [] # برای ذخیره ویدیو نهایی
num_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # تعداد فریم های ویدیو

for ii in range(num_frame):
    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)
    time_report = convertMillis(timestamp)
    ret, frame = cap.read() # خواندن یک فریم از ویدیو
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pilimg = Image.fromarray(frame)
    detections = detect_image(pilimg) # دادن فریم خوانده شده به شبکه دیتکشن

    img = np.array(pilimg)
    pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
    pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
    unpad_h = img_size - pad_y
    unpad_w = img_size - pad_x

    if detections is not None:
        tracked_objects = mot_tracker.update(detections.cpu()) # دادن خروجی دیتکشن به شبکه ترکینگ

        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)

        ########## نمایش جعبه های مرزی اشیاء و شناسه آن ها بر روی هر فریم ###########
        for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
            box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
            box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
            y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
            x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])

            color = colors[int(obj_id) % len(colors)]
            color = [i * 255 for i in color]
            cls = classes[int(cls_pred)]
            cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h), color, 1) # جعبه مرزی اطراف شیء
            cv2.rectangle(frame, (x1, y1-20), (x1+len(cls)*15, y1), color, -1) # جعبه مرزی بالای هر شی ء برای نمایش نوع کلاس  و شناسه شیء
            cv2.putText(frame, cls + "-" + str(int(obj_id)), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255,255,255), 1) # متن مربوط به اسم کلاس و شناسه

    Frames.append(frame) # ذخیره فریم های پردازش شده

###### ذخیره ویدیو نهایی ترک شده ######################
with imageio.get_writer("out.gif", mode="I") as writer:  
    for idx, frame in enumerate(Frames):
        writer.append_data(frame)